In [1]:
import pandas as pd
import json
import numpy as np

In [2]:
plot_features = pd.read_csv('plot_features.csv')
plot_features.head()

,movie_wikipedia_id,movie_freebase_id,movie_name,release_year,revenue,runtime,languages,countries,genres,plot_summary,...,char_count,avg_word_length,sentence_count,lexical_diversity,sentiment_polarity,topic,mood,target_audience,temporal_setting,location_setting
0,975900,/m/03vyhn,Ghosts of Mars,2001,14010832.0,98.0,"{""/m/02h40lc"": ""English Language""}","{""/m/09c7w0"": ""United States of America""}","{""/m/01jfsb"": ""Thriller"", ""/m/06n90"": ""Science...","Set in the second half of the 22nd century, th...",...,2181,6.109244,15,0.627451,-0.085095,"[""science fiction"", ""horror""]","[""dark"", ""dramatic""]","[""adults""]","[""future""]","[""fictional""]"
1,9363483,/m/0285_cd,White Of The Eye,1987,NaN,110.0,"{""/m/02h40lc"": ""English Language""}","{""/m/07ssc"": ""United Kingdom""}","{""/m/01jfsb"": ""Thriller"", ""/m/0glj9q"": ""Erotic...",A series of murders of rich young women throug...,...,3301,5.594915,36,0.542373,0.035867,"[""mystery"", ""drama""]","[""dark""]","[""adults""]","[""modern""]","[""real""]"
2,261236,/m/01mrr1,A Woman in Flames,1983,NaN,106.0,"{""/m/04306rv"": ""German Language""}","{""/m/0345h"": ""Germany""}","{""/m/07s9rl0"": ""Drama""}","Eva, an upper class housewife, becomes frustra...",...,2339,5.490610,24,0.582160,0.133259,"[""drama""]","[""dark""]","[""adults""]","[""modern""]","[""real""]"
3,18998739,/m/04jcqvw,The Sorcerer's Apprentice,2002,NaN,86.0,"{""/m/02h40lc"": ""English Language""}","{""/m/0hzlz"": ""South Africa""}","{""/m/0hqxf"": ""Family Film"", ""/m/01hmnh"": ""Fant...","Every hundred years, the evil Morgana returns...",...,870,5.337423,7,0.631902,0.040568,"[""fantasy"", ""adventure""]","[""inspirational""]","[""children""]","[""past""]","[""fictional""]"
4,6631279,/m/0gffwj,Little city,1997,NaN,93.0,"{""/m/02h40lc"": ""English Language""}","{""/m/09c7w0"": ""United States of America""}","{""/m/06cvj"": ""Romantic comedy"", ""/m/0hj3n0w"": ...","Adam, a San Francisco-based artist who works a...",...,1234,5.484444,9,0.626667,0.165202,"[""romance"", ""drama""]","[""romantic"", ""dramatic""]","[""adults""]","[""modern""]","[""real""]"


In [3]:
plot_features.columns

Index(['movie_wikipedia_id', 'movie_freebase_id', 'movie_name', 'release_year',
       'revenue', 'runtime', 'languages', 'countries', 'genres',
       'plot_summary', 'language', 'word_count', 'char_count',
       'avg_word_length', 'sentence_count', 'lexical_diversity',
       'sentiment_polarity', 'topic', 'mood', 'target_audience',
       'temporal_setting', 'location_setting'],
      dtype='object')

In [4]:
plot_features[['languages', 'countries', 'genres', 'topic', 'mood', 'target_audience',
       'temporal_setting', 'location_setting']] = plot_features[
            ['languages', 'countries', 'genres', 'topic', 'mood', 'target_audience',
       'temporal_setting', 'location_setting']
    ].map(json.loads, na_action='ignore')

In [5]:
plot_features[['languages', 'countries', 'genres']] = \
    plot_features[['languages', 'countries', 'genres']].map(
        lambda x: [y.lower() for y in x.values()], na_action='ignore')

In [6]:
set(y for x in plot_features['topic'].dropna() for y in x)

{'action',
 'adventure',
 'comedy',
 'conflict',
 'drama',
 'fantasy',
 'historical',
 'horror',
 'mystery',
 'romance',
 'science fiction',
 'tragedy'}

In [7]:
set(y for x in plot_features['topic'].dropna() for y in x) - \
    set(y for x in plot_features['genres'].dropna() for y in x)

{'conflict', 'historical', 'romance'}

Topics are just genres. Let's merge them into genres

In [8]:
plot_features['genres'] = (plot_features['genres'] + plot_features['topic'])\
    .map(lambda x: list(set(x)), na_action='ignore')
plot_features = plot_features.drop(columns='topic')

In [9]:
plot_features['language'].unique()

array(['en'], dtype=object)

In [10]:
plot_features = plot_features.drop(columns='language')

In [11]:
plot_features = plot_features[
    ~plot_features[['genres', 'mood', 'target_audience', 'temporal_setting', 'location_setting']]\
    .isna().any(axis=1)]

In [12]:
mood_values = set(y for x in plot_features['mood'].dropna() for y in x)
target_audience_values = set(y for x in plot_features['target_audience'].dropna() for y in x)
temporal_setting_values = set(y for x in plot_features['temporal_setting'].dropna() for y in x)
location_setting_values = set(y for x in plot_features['location_setting'].dropna() for y in x)

In [13]:
def count_llm_features(series, values):
    count_dict = dict()
    for value in values:
        count_dict[value] = series.apply(lambda x: value in x).sum()
    return dict(sorted(count_dict.items(), key=lambda x: -x[1]))

In [14]:
count_llm_features(plot_features['mood'], mood_values)

{'dramatic': 12960,
 'dark': 10002,
 'lighthearted': 5110,
 'exciting': 3008,
 'inspirational': 2902,
 'romantic': 2216,
 'fantastical': 421}

In [15]:
count_llm_features(plot_features['target_audience'], target_audience_values)

{'adults': 19930,
 'families': 2127,
 'teenagers': 2048,
 'children': 1241,
 'elderly': 16}

In [16]:
count_llm_features(plot_features['temporal_setting'], temporal_setting_values)

{'modern': 16843, 'past': 7304, 'future': 562}

In [17]:
count_llm_features(plot_features['location_setting'], location_setting_values)

{'real': 20394, 'fictional': 4947}

In [24]:
def create_cooccurrence_matrix(genres_series, feature_series):
    """
    Create a co-occurrence matrix from two pandas series: one with genres and one with feature values.

    Parameters:
    genres_series (pd.Series): A series of lists where each list contains genres.
    feature_series (pd.Series): A series of lists where each list contains feature values.

    Returns:
    pd.DataFrame: A DataFrame representing the co-occurrence matrix of genres and feature values.
    """
    # Initialize a dictionary to hold the co-occurrence counts
    cooccurrence_dict = {}

    # Iterate over the series to populate the dictionary
    for genres, feature_values in zip(genres_series, feature_series):
        for genre in genres:
            if genre not in cooccurrence_dict:
                cooccurrence_dict[genre] = {}

            for feature_value in feature_values:
                if feature_value not in cooccurrence_dict[genre]:
                    cooccurrence_dict[genre][feature_value] = 0
                cooccurrence_dict[genre][feature_value] += 1

    # Convert the dictionary to a DataFrame
    cooccurrence_matrix = pd.DataFrame(cooccurrence_dict).fillna(0).astype(int)

    # Reorder the matrix so that rows are features and columns are genres
    cooccurrence_matrix = cooccurrence_matrix.T

    return cooccurrence_matrix

In [28]:
def get_top_k_values(matrix, k, axis=0):
    """
    For each row or column in the matrix, find the top k values and return them with their labels.

    Parameters:
    matrix (pd.DataFrame): The co-occurrence matrix.
    k (int): The number of top values to return.
    axis (int): 0 to find top values in each row, 1 for each column.

    Returns:
    dict: A dictionary with rows or columns as keys and a list of tuples as values,
          where each tuple contains the label and the value of the top k items.
    """
    top_k_values = {}
    # Transpose the matrix if we are looking for top values in each column
    if axis == 1:
        matrix = matrix.T

    # Iterate over the matrix rows/columns
    for label, values in matrix.iterrows():
        # Get the top k values with their labels
        top_k = values.nlargest(k).items()
        # Store them in the dictionary
        top_k_values[label] = list(top_k)

    return top_k_values

In [40]:
create_cooccurrence_matrix(plot_features['genres'], plot_features['mood'])

,dark,dramatic,exciting,inspirational,lighthearted,romantic,fantastical
science fiction,949,799,811,123,258,59,180
space western,3,4,7,2,1,1,3
thriller,3380,2665,937,110,188,131,48
adventure,974,1855,1744,798,1511,267,285
action,2620,2827,2104,298,572,171,116
...,...,...,...,...,...,...,...
the netherlands in world war ii,0,1,0,1,0,0,0
period horror,1,1,0,0,0,0,0
revisionist fairy tale,0,1,2,1,1,0,1
film-opera,0,0,0,0,1,0,0


In [44]:
# get_top_k_values(create_cooccurrence_matrix(plot_features['genres'], plot_features['mood']), 5, 0)